# 03 - Federated Learning

## Defines

Define the available types of federated learning.

 - 'STRATIFIED': Stratified sampling of the data. The data is split into a number of shards, and each shard is assigned to a client. The data is split in a stratified manner, meaning that the distribution of the labels is approximately the same in each shard.
 - 'MISSING_1_ATTACK' - Each client is assigned a shard of data, each shard is missing one of the attack labels. Other clients in the network are exposed to the attack label, but the specific client is not. This demonstrates the ability of federated learning to protect against unknown attacks.
 - '1_ATTACK_ONLY' - Each client is assigned a shard of data, each shard contains only one of the attack labels.
 - 'HALF_BENIGN_ONLY' - Half of the clients are exposed to Benign data only, the other half are exposed to all data.


In [49]:
### THIS SECTION NEEDS TO BE SET TO DETERMINE WHICH CONFIGURATION METHOD TO UTILISE

SPLIT_AVAILABLE_METHODS = ['STRATIFIED','MISSING_1_ATTACK', '1_ATTACK_ONLY', 'HALF_BENIGN_ONLY' ]
METHOD = 'HALF_BENIGN_ONLY'
NUM_OF_STRATIFIED_CLIENTS = 10  # only applies to stratified method
NUM_OF_ROUNDS = 10              # Number of FL rounds


The above test method in conjunction with the below classification selection will determine the number of clients.

EG: 
`STRATIFIED` with:
 - `ALL TYPES` - Results in `NUM_OF_STRATIFIED_CLIENTS` clients. Each client will have a stratified sample of the data.

`MISSING_1_ATTACK` with:
 - `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 32 attack labels.
 - `group_classifier` - Results in 7 clients. Each client will have benign traffic and 6 attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels.

`1_ATTACK_ONLY` with:
 - `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 1 attack label.
 - `group_classifier` - Results in 7 clients. Each client will have benign traffic and 1 attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels. - SAME AS MISSING_1_ATTACK for binary classifier

`HALF_BENIGN_ONLY` with:
 - `individual_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 33 malicious attack labels.
 - `group_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 7 malicious attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels. - SAME AS MISSING_1_ATTACK for binary classifier


In [50]:
individual_classifier = False
group_classifier = False
binary_classifier = True


Include the defines for the dataframe columns and the attack labels and their mappings

In [51]:
from enum import Enum
from includes import *

##  Imports

In [52]:
%%capture
%pip install flwr[simulation] torch torchvision matplotlib sklearn openml

In [53]:
import os
import pandas as pd
import numpy as np
import flwr as fl
from tqdm import tqdm
import warnings
#warnings.filterwarnings('ignore')

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split


In [54]:
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

flwr 1.4.0
numpy 1.23.5
torch 2.0.1+cpu
Training on cpu


## Load the Dataset

In [55]:
DATASET_DIRECTORY = '../datasets/CICIoT2023/'

## Training data

Either read the training pickle file if it exists, or process the dataset from scratch.

In [56]:
# Check to see if the file 'training_data.pkl' exists in the directory. If it does, load it. If not, print an error.
if os.path.isfile('training_data.pkl'):
    print("File exists, loading data...")
    train_df = pd.read_pickle('training_data.pkl')
    print("Training data loaded from pickle file.")

else:
    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    # Print the number of files in each set
    print('Training sets: {}'.format(len(training_sets)))
    print('Test sets: {}'.format(len(test_sets)))

    # ######################
    # # HACK TEMP CODE
    # ######################
    # # Set training_sets to the last entry of training_sets
    # training_sets = training_sets[-33:]
    # print(f"HACK TO REPLICATE ORIGINAL AUTHORS CODE WITH ONE FILE TRAIN - {training_sets}")
    # #####################
    # # HACK END TEMP CODE
    # ######################

    # Concatenate all training sets into one dataframe
    dfs = []
    print("Reading training data...")
    for train_set in tqdm(training_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + train_set)
        dfs.append(df_new)
    train_df = pd.concat(dfs, ignore_index=True)

    # Complete training data set size
    print("Complete training data size: {}".format(train_df.shape))

    # Map y column to the dict_34_classes values - The pickle file already has this done.
    train_df['label'] = train_df['label'].map(dict_34_classes)

    # The training data is the 80% of the CSV files in the dataset. The test data is the remaining 20%.
    # The Ray Federated learning mechanism cannot cope with all of the 80% training data, so we will split
    # the training data using test_train_split. The test data will be ignored as we will use all the data 
    # from the train_sets files as our training data to keep parity with the original authors code.
    # 
    # By using a subset of the training data split this way, we can have a randomised selection of data
    # from all the training CSV files, stratified by the attack types.
    
    # Percentage of original training data to use.
    TRAIN_SIZE = 0.0125
    
    print(f"Splitting the data into {TRAIN_SIZE*100}%")
    
    X_train, X_test, y_train, y_test = train_test_split(train_df[X_columns], train_df[y_column], test_size= (1 - TRAIN_SIZE), random_state=42, stratify=train_df[y_column])

    # Recombine X_train, and y_train into a dataframe
    train_df = pd.concat([X_train, y_train], axis=1)

    # Clean up unused variables

    del X_train, y_train, X_test, y_test
    
    # Save the output to a pickle file
    print("Writing training data to pickle file...")
    train_df.to_pickle('training_data.pkl')

print("Training data size: {}".format(train_df.shape))


File exists, loading data...
Training data loaded from pickle file.
Training data size: (454330, 47)


In [57]:
# show the unique values counts in the label column for train_df
print("Counts of attacks in train_df:")
print(train_df['label'].value_counts())

Counts of attacks in train_df:
6     70072
4     52698
5     43768
2     39835
3     39480
1     39364
7     35032
13    32300
15    25994
14    19737
0     10686
17     9647
19     8665
18     7319
10     4397
26     2995
9      2798
8      2780
25     1739
24     1308
21      957
22      798
16      701
23      364
12      279
11      227
33      126
27       58
32       53
31       51
29       37
28       31
20       22
30       12
Name: label, dtype: int64


In [58]:
train_df

flow_duration  Header_Length  Protocol Type  Duration        Rate  \
29467977       0.000000         172.92           6.00     65.69    4.552125   
8309328        0.000000           0.00           1.00     64.00    0.400035   
12088899       5.799076         108.00           6.00     64.00    0.344883   
2403491        0.146236         522.40           9.80     81.00   32.454440   
33786435       0.000000          54.00           6.00     64.00   13.801025   
...                 ...            ...            ...       ...         ...   
32944878       0.000000           0.00           1.00     64.00  108.962772   
23809291       0.000000          54.00           6.00     64.00   17.262038   
12491659       1.159555          79.38           5.94     63.36    0.620923   
34788276       0.000000           0.00           1.00     64.00    3.148525   
23655092       1.017786          73.44           6.00     64.00    0.550216   

               Srate  Drate  fin_flag_number  syn_flag_number  \
29467977    4.552125    0.0              0.0              0.0   
8309328     0.400035    0.0              0.0              0.0   
12088899    0.344883    0.0              0.0              0.0   
2403491    32.454440    0.0              0.0              0.0   
33786435   13.801025    0.0              0.0              1.0   
...              ...    ...              ...              ...   
32944878  108.962772    0.0              0.0              0.0   
23809291   17.262038    0.0              0.0              0.0   
12491659    0.620923    0.0              0.0              1.0   
34788276    3.148525    0.0              0.0              0.0   
23655092    0.550216    0.0              0.0              1.0   

          rst_flag_number  ...         Std  Tot size           IAT  Number  \
29467977              0.0  ...    6.433546    172.92  8.306406e+07     9.5   
8309328               0.0  ...    0.000000     42.00  8.314956e+07     9.5   
12088899              0.0  ...    0.000000     54.00  8.294671e+07     9.5   
2403491               0.0  ...  129.331845    163.50  3.758001e-03     5.5   
33786435              0.0  ...    0.000000     54.00  8.308929e+07     9.5   
...                   ...  ...         ...       ...           ...     ...   
32944878              0.0  ...    0.000000     42.00  8.315005e+07     9.5   
23809291              0.0  ...    0.000000     54.00  8.307206e+07     9.5   
12491659              0.0  ...    0.273154     54.06  8.336239e+07     9.5   
34788276              0.0  ...    0.000000     42.00  8.312780e+07     9.5   
23655092              0.0  ...    0.000000     54.00  8.336214e+07     9.5   

           Magnitue      Radius    Covariance  Variance  Weight  label  
29467977  18.498183    9.106583    233.739318      0.19  141.55      5  
8309328    9.165151    0.000000      0.000000      0.00  141.55      6  
12088899  10.392305    0.000000      0.000000      0.00  141.55     15  
2403491   19.087867  182.902849  21525.530784      0.90   38.50     25  
33786435  10.392305    0.000000      0.000000      0.00  141.55      3  
...             ...         ...           ...       ...     ...    ...  
32944878   9.165151    0.000000      0.000000      0.00  141.55      6  
23809291  10.392305    0.000000      0.000000      0.00  141.55      5  
12491659  10.401036    0.386753      0.483918      0.16  141.55      7  
34788276   9.165151    0.000000      0.000000      0.00  141.55      6  
23655092  10.392305    0.000000      0.000000      0.00  141.55      7  

[454330 rows x 47 columns]

---
## Test Data
Concat the test data into a single dataframe

In [59]:
# Check to see if the file 'test_data.pkl' exists in the directory. If it does, load it. If not, print an error.
testing_data_pickle_file = 'testing_data.pkl'

if os.path.isfile(testing_data_pickle_file):
    print(f"File {testing_data_pickle_file} exists, loading data...")
    test_df = pd.read_pickle(testing_data_pickle_file)
    print("Test data loaded from pickle file.")

else:
    print(f"File {testing_data_pickle_file} does not exist, constructing data...")

    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    ############################################
    ############################################
    # HACK - Make things quicker for now
    ############################################
    ############################################

    # test_sets = df_sets[int(len(df_sets)*.95):]
    
    # # Set training_sets to the last entry of training_sets
    # test_sets = test_sets[-2:]
    
    ############################################
    ############################################
    # END HACK 
    ############################################
    ############################################

    # Print the number of files in each set
    print('Test sets: {}'.format(len(test_sets)))
    
    # Concatenate all testing sets into one dataframe
    dfs = []
    print("Reading test data...")
    for test_set in tqdm(test_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + test_set)
        dfs.append(df_new)
    test_df = pd.concat(dfs, ignore_index=True)

    # Map y column to the dict_34_classes values - The pickle file already has this done.
    test_df['label'] = test_df['label'].map(dict_34_classes)

    # Save the output to a pickle file
    print(f"Writing test data to pickle file {testing_data_pickle_file}...")
    test_df.to_pickle(testing_data_pickle_file)

print("Testing data size: {}".format(test_df.shape))

File testing_data.pkl exists, loading data...
Test data loaded from pickle file.
Testing data size: (10340161, 47)


In [60]:
print("Number of rows in train_df: {}".format(len(train_df)))
print("Number of rows in test_df: {}".format(len(test_df)))

train_size = len(train_df)
test_size = len(test_df)

Number of rows in train_df: 454330
Number of rows in test_df: 10340161


---
# Scale the test and train data

### Scale the training data input features

In [61]:
scaler = StandardScaler()
train_df[X_columns] = scaler.fit_transform(train_df[X_columns])

### Scale the testing data input features

In [62]:
test_df[X_columns] = scaler.fit_transform(test_df[X_columns])

---
# Define the classification problem - (2 classes, 8 classes or 34 classes)
Change the following cell to select the classification type

If the METHOD == STRATIFIED, then we can use any classifier
If the METHOD == ATTACK_GROUP then we must use Group Classifier.

In [63]:

class_size_map = {2: "Binary", 8: "Group", 34: "Individual"}

if group_classifier:
    print("Group 8 Class Classifier... - Adjusting labels in test and train dataframes")
    # Map y column to the dict_7_classes values
    test_df['label'] = test_df['label'].map(dict_8_classes)
    train_df['label'] = train_df['label'].map(dict_8_classes)
    class_size = "8"      
    
elif binary_classifier:
    print("Binary 2 Class Classifier... - Adjusting labels in test and train dataframes")
    # Map y column to the dict_2_classes values
    test_df['label'] = test_df['label'].map(dict_2_classes)
    train_df['label'] = train_df['label'].map(dict_2_classes)
    class_size = "2"

else:
    print ("Individual 34 Class classifier... - No adjustments to labels in test and train dataframes")
    class_size = "34"


Binary 2 Class Classifier... - Adjusting labels in test and train dataframes


---
# Split the Training Data into partitions for the Federated Learning clients depending on the test required
As a reminder:

`STRATIFIED` with:
 - `ALL TYPES` - Results in `NUM_OF_STRATIFIED_CLIENTS` clients. Each client will have a stratified sample of the data.

`MISSING_1_ATTACK` with:
 - `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 32 attack labels.
 - `group_classifier` - Results in 7 clients. Each client will have benign traffic and 6 attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels.

`1_ATTACK_ONLY` with:
 - `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 1 attack label.
 - `group_classifier` - Results in 7 clients. Each client will have benign traffic and 1 attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels. - SAME AS MISSING_1_ATTACK for binary classifier

`HALF_BENIGN_ONLY` with:
 - `individual_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 33 malicious attack labels.
 - `group_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 7 malicious attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels. - SAME AS MISSING_1_ATTACK for binary classifier


In [64]:
from sklearn.model_selection import StratifiedKFold

# Define fl_X_train and fl_y_train
fl_X_train = []
fl_y_train = []

client_df = pd.DataFrame()

if METHOD == 'STRATIFIED':
    print(f"{Colours.YELLOW.value}STRATIFIED METHOD{Colours.NORMAL.value} with {class_size} class classifier")
    # We are going to split the training data into 'NUM_OF_STRATIFIED_CLIENTS' smaller groups using StratifiedKFold
    skf = StratifiedKFold(n_splits=NUM_OF_STRATIFIED_CLIENTS, shuffle=True, random_state=42)
    for train_index, test_index in skf.split(train_df[X_columns], train_df[y_column]):
        fl_X_train.append(train_df[X_columns].iloc[test_index])
        fl_y_train.append(train_df[y_column].iloc[test_index])

elif METHOD == 'MISSING_1_ATTACK':
    print(f"{Colours.YELLOW.value}MISSING_1_ATTACK METHOD{Colours.NORMAL.value} with {class_size} class classifier")

    if individual_classifier or group_classifier:
        # Set the number of splits required to the number of classes - 1
        num_splits = int(class_size) - 1
    else:
        # For binary classifier, set the number of splits to 10
        num_splits = 10

    skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

    # When creating the clients, we will remove one attack class from the training data
    # For the binary classifier, evey other client will have the benign class removed
    for i, (train_index, test_index) in enumerate(skf.split(train_df[X_columns], train_df[y_column])):
        if binary_classifier:
            print(f"i: {i} = i % 2 = {i % 2}")
            if i % 2 == 0:
                print("Benign only")
                # Create a new dataframe for the client data with only benign traffic
                client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != 1]], ignore_index=True)
                fl_X_train.append(client_df[X_columns])
                fl_y_train.append(client_df[y_column])
            else:
                print("Both")
                # Create a new dataframe for the client data
                fl_X_train.append(train_df[X_columns].iloc[test_index])
                fl_y_train.append(train_df[y_column].iloc[test_index])
        else:
            # Create a new dataframe for the client data
            client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != i+1]], ignore_index=True)
            fl_X_train.append(client_df[X_columns])
            fl_y_train.append(client_df[y_column])

elif METHOD == '1_ATTACK_ONLY':
    print(f"{Colours.YELLOW.value}1_ATTACK_ONLY METHOD{Colours.NORMAL.value} with {class_size} class classifier")
    # Each client only has one attack class in their training data along with the Benign data
    
    if individual_classifier or group_classifier:
        # Set the number of splits required to the number of classes - 1
        num_splits = int(class_size) - 1
    else:
        # For binary classifier, set the number of splits to 10
        num_splits = 10

    skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

    # When creating the clients, we will only add the benign data and the attack class for that client
    for i, (train_index, test_index) in enumerate(skf.split(train_df[X_columns], train_df[y_column])):
        if binary_classifier:
            print(f"i: {i} = i % 2 = {i % 2}")
            if i % 2 == 0:
                print("Benign only")
                # Create a new dataframe for the client data with only benign traffic
                client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != 1]], ignore_index=True)
                fl_X_train.append(client_df[X_columns])
                fl_y_train.append(client_df[y_column])
            else:
                print("Both")
                # Create a new dataframe for the client data
                fl_X_train.append(train_df[X_columns].iloc[test_index])
                fl_y_train.append(train_df[y_column].iloc[test_index])
        else:
            # Create a new dataframe for the client data
            client_df = pd.concat([train_df.iloc[test_index][(train_df[y_column] == 0) | (train_df[y_column] == i+1)]], ignore_index=True)
            fl_X_train.append(client_df[X_columns])
            fl_y_train.append(client_df[y_column])

elif METHOD == 'HALF_BENIGN_ONLY':
    print(f"{Colours.YELLOW.value}HALF_BENIGN_ONLY METHOD{Colours.NORMAL.value} with {class_size} class classifier")

    num_splits = 10

    # Split into 10 client data
    skf = StratifiedKFold(n_splits=NUM_OF_STRATIFIED_CLIENTS, shuffle=True, random_state=42)

    # For i % 2 == 0, add only benign data
    # For i % 2 == 1, add all data
    for i, (train_index, test_index) in enumerate(skf.split(train_df[X_columns], train_df[y_column])):
        if i % 2 == 0:
            print("Benign only")
            # Create a new dataframe for the client data with only benign traffic
            client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] == 0]], ignore_index=True)
            fl_X_train.append(client_df[X_columns])
            fl_y_train.append(client_df[y_column])
        else:
            print("All Classes")
            fl_X_train.append(train_df[X_columns].iloc[test_index])
            fl_y_train.append(train_df[y_column].iloc[test_index])
else:
    print(f"{Colours.RED.value}ERROR: Method {METHOD} not recognised{Colours.NORMAL.value}")
    exit()



STRATIFIED METHOD with 2 class classifier


In [65]:
NUM_OF_CLIENTS = len(fl_X_train)

for i in range(len(fl_X_train)):
    # Show the unique values in the y column
    (f"Client ID: {i}")
    print(f"fl_X_train[{i}].shape: {fl_X_train[i].shape}")  
    print(f"fl_y_train[{i}].value_counts():\n{fl_y_train[i].value_counts()}")
    print(f"fl_y_train[{i}].unique(): {fl_y_train[i].unique()}\n")

# Check that fl_X_train[0] and fl_X_train[1] contain different data
print(f"fl_X_train[0].equals(fl_X_train[1]): {fl_X_train[0].equals(fl_X_train[1])}")

fl_X_train[0].shape: (45433, 46)
fl_y_train[0].value_counts():
1    44365
0     1068
Name: label, dtype: int64
fl_y_train[0].unique(): [1 0]

fl_X_train[1].shape: (45433, 46)
fl_y_train[1].value_counts():
1    44365
0     1068
Name: label, dtype: int64
fl_y_train[1].unique(): [1 0]

fl_X_train[2].shape: (45433, 46)
fl_y_train[2].value_counts():
1    44365
0     1068
Name: label, dtype: int64
fl_y_train[2].unique(): [1 0]

fl_X_train[3].shape: (45433, 46)
fl_y_train[3].value_counts():
1    44365
0     1068
Name: label, dtype: int64
fl_y_train[3].unique(): [1 0]

fl_X_train[4].shape: (45433, 46)
fl_y_train[4].value_counts():
1    44364
0     1069
Name: label, dtype: int64
fl_y_train[4].unique(): [1 0]

fl_X_train[5].shape: (45433, 46)
fl_y_train[5].value_counts():
1    44364
0     1069
Name: label, dtype: int64
fl_y_train[5].unique(): [1 0]

fl_X_train[6].shape: (45433, 46)
fl_y_train[6].value_counts():
1    44364
0     1069
Name: label, dtype: int64
fl_y_train[6].unique(): [1 0]

fl_X_t

Prepare an output directory where we can store the results of the federated learning

In [66]:
# Create an "Output" directory if it doesnt exist already
if not os.path.exists("Output"):
    os.makedirs("Output")

sub_dir_name = f"train_size-{train_size}_test_size-{test_size}"

# if sub_dir_name does not exist, create it
if not os.path.exists(f"Output/{sub_dir_name}"):
    os.makedirs(f"Output/{sub_dir_name}")

test_directory_name = f"{METHOD}_Classifier-{class_size}_Clients-{NUM_OF_CLIENTS}"

# Create an "Output/{METHOD}-{NUM_OF_CLIENTS}-{NUM_OF_ROUNDS}" directory if it doesnt exist already
if not os.path.exists(f"Output/{sub_dir_name}/{test_directory_name}"):
    os.makedirs(f"Output/{sub_dir_name}/{test_directory_name}")

# Ensure the directory is empty
for file in os.listdir(f"Output/{sub_dir_name}/{test_directory_name}"):
    file_path = os.path.join(f"Output/{sub_dir_name}/{test_directory_name}", file)
    if os.path.isfile(file_path):
        os.unlink(file_path)

# Original training size is the sum of all the fl_X_train sizes
original_training_size = 0
for i in range(len(fl_X_train)):
    original_training_size += fl_X_train[i].shape[0]

# Write this same info to the output directory/Class Split Info.txt
with open(f"Output/{sub_dir_name}/{test_directory_name}/Class Split Info.txt", "w") as f:
    for i in range(len(fl_X_train)):
        f.write(f"Client ID: {i}\n")
        f.write(f"fl_X_train.shape: {fl_X_train[i].shape}\n")
        f.write(f"Training data used {original_training_size}")
        f.write(f"fl_y_train.value_counts():\n{fl_y_train[i].value_counts()}\n")
        f.write(f"fl_y_train.unique(): {fl_y_train[i].unique()}\n\n")

### Convert the training dataset

In [67]:
# Convert the testing daya to X_test and y_test ndarrays
X_test = test_df[X_columns].to_numpy()
y_test = test_df[y_column].to_numpy()

In [68]:
num_unique_classes = len(train_df[y_column].unique())

train_df_shape = train_df.shape
test_df_shape = test_df.shape

# We are now done with the train_df and test_df dataframes, so we can delete them to free up memory
del train_df
del test_df
del client_df

---
### Data check

In [69]:
print("NUM_CLIENTS:", NUM_OF_CLIENTS)

print("NUM_ROUNDS:", NUM_OF_ROUNDS)
print()


print("Original training size: {}".format(original_training_size))


print("Checking training data split groups")
for i in range(len(fl_X_train)):
    print(i, ":", "X Shape", fl_X_train[i].shape, "Y Shape", fl_y_train[i].shape)


# Print the sizes of X_test and y_test
print("\nChecking testing data")
print("X_test size: {}".format(X_test.shape))
print("y_test size: {}".format(y_test.shape))

print("\nDeploy Simulation")

NUM_CLIENTS: 10
NUM_ROUNDS: 10

Original training size: 454330
Checking training data split groups
0 : X Shape (45433, 46) Y Shape (45433,)
1 : X Shape (45433, 46) Y Shape (45433,)
2 : X Shape (45433, 46) Y Shape (45433,)
3 : X Shape (45433, 46) Y Shape (45433,)
4 : X Shape (45433, 46) Y Shape (45433,)
5 : X Shape (45433, 46) Y Shape (45433,)
6 : X Shape (45433, 46) Y Shape (45433,)
7 : X Shape (45433, 46) Y Shape (45433,)
8 : X Shape (45433, 46) Y Shape (45433,)
9 : X Shape (45433, 46) Y Shape (45433,)

Checking testing data
X_test size: (10340161, 46)
y_test size: (10340161,)

Deploy Simulation


----
# Federated Learning
## Import the libraries and print the versions

In [70]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout



Define the Client and Server code

In [71]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

print('scikit-learn {}.'.format(sklearn.__version__))
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("tf", tf.__version__)
# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

import datetime

client_evaluations = []

class NumpyFlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, train_data, train_labels):
        self.model = model
        self.cid = cid
        self.train_data = train_data
        self.train_labels = train_labels

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Training...")
        self.model.fit(self.train_data, self.train_labels, epochs=5, batch_size=32)
        print ("Client ", self.cid, "Training complete...")
        return self.model.get_weights(), len(self.train_data), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Evaluating...")
        loss, accuracy = self.model.evaluate(self.train_data, self.train_labels, batch_size=32)
        print(f"{Colours.YELLOW.value}Client {self.cid} evaluation complete - Accuracy: {accuracy:.6f}, Loss: {loss:.6f}{Colours.NORMAL.value}")

        # Write the same message to the "Output/{cid}_Evaluation.txt" file
        with open(f"Output/{sub_dir_name}/{test_directory_name}/{self.cid}_Evaluation.txt", "a") as f:
            f.write(f"{datetime.datetime.now()} - Client {self.cid} evaluation complete - Accuracy: {accuracy:.6f}, Loss: {loss:.6f}\n")

            # Close the file
            f.close()

        return loss, len(self.train_data), {"accuracy": accuracy}
    
    def predict(self, incoming):
        prediction = np.argmax( self.model.predict(incoming) ,axis=1)
        return prediction

def client_fn(cid: str) -> NumpyFlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    #model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
    #model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

    print ("Client ID:", cid)

    model = Sequential([
      #Flatten(input_shape=(79,1)),
      Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
      Dense(50, activation='relu'),  
      Dense(25, activation='relu'),  
      Dense(num_unique_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

   
    partition_id = int(cid)
    X_train_c = fl_X_train[partition_id]
    y_train_c = fl_y_train[partition_id]

    # Create a  single Flower client representing a single organization
    return NumpyFlowerClient(cid, model, X_train_c, y_train_c)


from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
eval_count = 0

def get_evaluate_fn(server_model):
    global eval_count
    """Return an evaluation function for server-side evaluation."""
    # The `evaluate` function will be called after every round
    
    
    def evaluate(server_round, parameters, config):
        global eval_count
        
        # Update model with the latest parameters
        server_model.set_weights(parameters)
        print (f"Server Evaluating... Evaluation Count:{eval_count}")
        loss, accuracy = server_model.evaluate(X_test, y_test)
        
        y_pred = server_model.predict(X_test)
        print ("Prediction: ", y_pred, y_pred.shape)
        #cmatrix = confusion_matrix(y_test, np.rint(y_pred))
        #print ("confusion_matrix:", cmatrix, cmatrix.shape)
                        
        print(f"{Colours.YELLOW.value}Server evaluation complete - Accuracy: {accuracy:.4f}, Loss: {loss:.4f}{Colours.NORMAL.value}")

        # Write the same message to the "Output/Server_Evaluation.txt" file
        with open(f"Output/{sub_dir_name}/{test_directory_name}/Server_Evaluation.txt", "a") as f:
            f.write(f"{datetime.datetime.now()} - {server_round} : Server evaluation complete - Accuracy: {accuracy:.4f}, Loss: {loss:.4f}\n")

            # Close the file
            f.close()
        
        np.save("y_pred-" + str(eval_count) + ".npy", y_pred)
        #np.save("cmatrix-" + str(eval_count) + ".npy", cmatrix)
        eval_count = eval_count + 1
        
        return loss, {"accuracy": accuracy}
    return evaluate



server_model = Sequential([
    #Flatten(input_shape=(79,1)),
    Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
    Dense(50, activation='relu'),  
    Dense(25, activation='relu'),  
    Dense(num_unique_classes, activation='softmax')
])


server_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2, #10,
        min_evaluate_clients=2, #5,
        min_available_clients=2, #10,
        evaluate_fn=get_evaluate_fn(server_model),
        #evaluate_metrics_aggregation_fn=weighted_average,
)

scikit-learn 1.2.2.
flwr 1.4.0
numpy 1.23.5
tf 2.12.0


In [72]:
%%time
print (f"{Colours.YELLOW.value}\nDeploy simulation... Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier")
print (f"Number of Clients = {NUM_OF_CLIENTS}\n")
print (f"Writing output to: {sub_dir_name}/{test_directory_name}\n{Colours.NORMAL.value}")

# Output the same information to the Output/Run_details.txt file
with open(f"Output/{sub_dir_name}/{test_directory_name}/Run_details.txt", "a") as f:
    f.write(f"{datetime.datetime.now()} - Deploy simulation... Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier\n")
    f.write(f"{datetime.datetime.now()} - Number of Clients = {NUM_OF_CLIENTS}\n")

    # Write Original train_df size
    f.write(f"{datetime.datetime.now()} - Original train_df size: {train_df_shape}\n")

    # Write the training data split groups
    for i in range(len(fl_X_train)):
        f.write(f"{datetime.datetime.now()} - {i}: X Shape {fl_X_train[i].shape}, Y Shape {fl_y_train[i].shape}\n")

    # Write the testing data
    f.write(f"{datetime.datetime.now()} - X_test size: {X_test.shape}\n")
    f.write(f"{datetime.datetime.now()} - y_test size: {y_test.shape}\n")
    
# close the file
f.close()

start_time = datetime.datetime.now()

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_OF_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_OF_ROUNDS),
    strategy=strategy,
)

end_time = datetime.datetime.now()
print("Total time taken: ", end_time - start_time)

print (f"{Colours.YELLOW.value} SIMULATION COMPLETE. Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier")
print (f"Number of Clients = {NUM_OF_CLIENTS}{Colours.NORMAL.value}\n")

# Output the same information to the Output/Run_details.txt file
with open(f"Output/{sub_dir_name}/{test_directory_name}/Run_details.txt", "a") as f:
    f.write(f"{datetime.datetime.now()} - SIMULATION COMPLETE. Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier\n")
    f.write(f"{datetime.datetime.now()} - Total time taken: {end_time - start_time}\n")

INFO flwr 2023-07-13 07:35:21,920 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)



Deploy simulation... Method = STRATIFIED - Binary (2) Classifier
Number of Clients = 10

Writing output to: train_size-454330_test_size-10340161/STRATIFIED_Classifier-2_Clients-10



(launch_and_evaluate pid=13520) Client ID: 9 [repeated 4x across cluster]
(launch_and_evaluate pid=13520)  793/1420 [===============>..............] - ETA: 0s - loss: 0.0312 - accuracy: 0.9878 [repeated 9x across cluster]
(launch_and_evaluate pid=13520) 1131/1420 [======================>.......] - ETA: 0s - loss: 0.0309 - accuracy: 0.9878 [repeated 9x across cluster]
(launch_and_evaluate pid=13520) 1167/1420 [=======================>......] - ETA: 0s - loss: 0.0310 - accuracy: 0.9878 [repeated 6x across cluster]
(launch_and_evaluate pid=13520)  [repeated 118x across cluster]
(launch_and_evaluate pid=23200) 1227/1420 [========================>.....] - ETA: 0s - loss: 0.0318 - accuracy: 0.9885 [repeated 7x across cluster]
(launch_and_evaluate pid=13520) 1275/1420 [=========================>....] - ETA: 0s - loss: 0.0315 - accuracy: 0.9876 [repeated 6x across cluster]
(launch_and_evaluate pid=13520) 1310/1420 [==========================>...] - ETA: 0s - loss: 0.0314 - accuracy: 0.9877 [re

2023-07-13 07:35:32,434	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-07-13 07:35:36,467 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 12.0, 'node:127.0.0.1': 1.0, 'memory': 11621872436.0, 'object_store_memory': 5810936217.0, 'GPU': 1.0}
INFO flwr 2023-07-13 07:35:36,468 | server.py:86 | Initializing global parameters
INFO flwr 2023-07-13 07:35:36,469 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2023-07-13 07:35:43,922 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-07-13 07:35:43,923 | server.py:88 | Evaluating initial parameters


(launch_and_get_parameters pid=14840) Client ID: 8
Server Evaluating... Evaluation Count:0
323131/323131 [==============================] - 249s 772us/step


...

Server Evaluating... Evaluation Count:10
323131/323131 [==============================] - 269s 831us/step


INFO flwr 2023-07-13 09:44:02,875 | server.py:119 | fit progress: (10, 0.02126791700720787, {'accuracy': 0.9933499097824097}, 7066.682228199992)
DEBUG flwr 2023-07-13 09:44:02,876 | server.py:168 | evaluate_round 10: strategy sampled 5 clients (out of 10)


Prediction:  [[3.4326790e-23 1.0000000e+00]
 [9.3653885e-18 1.0000000e+00]
 [2.5980499e-01 7.4019504e-01]
 ...
 [7.4470707e-10 1.0000000e+00]
 [2.4229750e-12 1.0000000e+00]
 [5.1353033e-15 1.0000000e+00]] (10340161, 2)
Server evaluation complete - Accuracy: 0.9933, Loss: 0.0213
(launch_and_evaluate pid=27464) Client ID: 6
(launch_and_fit pid=27464) Client ID: 6
(launch_and_fit pid=27464)  845/1420 [================>.............] - ETA: 1s - loss: 0.0135 - accuracy: 0.9942 [repeated 6x across cluster]
(launch_and_fit pid=27464)  880/1420 [=================>............] - ETA: 1s - loss: 0.0137 - accuracy: 0.9940 [repeated 5x across cluster]
(launch_and_fit pid=27464)  946/1420 [==================>...........] - ETA: 1s - loss: 0.0139 - accuracy: 0.9939 [repeated 5x across cluster]
(launch_and_fit pid=27464)  970/1420 [===================>..........] - ETA: 1s - loss: 0.0141 - accuracy: 0.9937 [repeated 4x across cluster]
(launch_and_fit pid=27464) 1020/1420 [====================>.....

DEBUG flwr 2023-07-13 09:44:05,398 | server.py:182 | evaluate_round 10 received 5 results and 0 failures
INFO flwr 2023-07-13 09:44:05,399 | server.py:147 | FL finished in 7069.206771500001
INFO flwr 2023-07-13 09:44:05,400 | app.py:218 | app_fit: losses_distributed [(1, 0.01831797994673252), (2, 0.01668887920677662), (3, 0.016572396084666253), (4, 0.015234833396971226), (5, 0.015562286227941513), (6, 0.0153903191909194), (7, 0.014799277298152447), (8, 0.014341126382350921), (9, 0.015447058714926243), (10, 0.015311380103230476)]
INFO flwr 2023-07-13 09:44:05,401 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-07-13 09:44:05,401 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-07-13 09:44:05,402 | app.py:221 | app_fit: losses_centralized [(0, 0.7927687168121338), (1, 0.019425245001912117), (2, 0.01820513978600502), (3, 0.017775995656847954), (4, 0.017621871083974838), (5, 0.017770841717720032), (6, 0.01838020607829094), (7, 0.018535353243350983), (8, 0.019

(launch_and_evaluate pid=14840) Client 0 evaluation complete - Accuracy: 0.993705, Loss: 0.013951
Total time taken:  2:08:43.487250
 SIMULATION COMPLETE. Method = STRATIFIED - Binary (2) Classifier
Number of Clients = 10

CPU times: total: 2h 11min 29s
Wall time: 2h 8min 43s
